# Construindo um RAG com LangChain e Gemini

Este notebook demonstra como construir um sistema **RAG (Retrieval-Augmented Generation)** utilizando o **Google Gemini**, `LangChain` e `ChromaDB`.  
A ideia é carregar documentos em PDF, criar embeddings, armazená-los em um banco vetorial e depois consultar as informações de forma interativa.


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, Markdown

## 1. Carregar variáveis de ambiente e definir diretórios

Aqui carregamos a **API Key** do Gemini (armazenada no `.env`) e definimos o diretório onde o banco vetorial (ChromaDB) será persistido.


In [ ]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
persist_directory = "./chroma_db"

## 2. Ingestão de Dados (PDF → Texto)

1. Carregamos o documento PDF com o `PyPDFLoader`.  
2. Dividimos o texto em pedaços menores (`chunks`) para facilitar a indexação e a recuperação de contexto posterior.


In [ ]:

pdf_path = r"sample_apostila_direito_adm.pdf"  # <<-- Altere para o caminho do seu arquivo PDF
loader = PyPDFLoader(pdf_path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = loader.load_and_split(text_splitter)

## 3. Criar Embeddings e Banco Vetorial (ChromaDB)

- Utilizamos o modelo de embeddings do **Gemini** (`models/embedding-001`).  
- Armazenamos os vetores em um banco de dados local (`ChromaDB`) para consultas rápidas.


In [ ]:
embeddings_model = GoogleGenerativeAIEmbeddings(google_api_key=GEMINI_API_KEY, model="models/embedding-001")
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=persist_directory
)

## 4. Configurar o Recuperador e o Prompt

- `retriever`: responsável por buscar os trechos mais relevantes no banco vetorial.  
- `ChatPromptTemplate`: define a estrutura do prompt que será enviado ao modelo, incluindo o **contexto** e a **pergunta** do usuário.


In [ ]:
retriever = vector_store.as_retriever()

# Definir o template do prompt
template = """
Use os seguintes pedaços de contexto para responder à pergunta no final.
Se você não souber a resposta, diga que não sabe.

Contexto: {context}

Pergunta: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

## 5. Inicializar o Modelo Gemini

Aqui usamos o `ChatGoogleGenerativeAI` com temperatura 0.7 para permitir respostas criativas, mas ainda consistentes.


In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, google_api_key=GEMINI_API_KEY)

## 6. Construir a Cadeia RAG (LCEL)

Combinamos:
1. Recuperação de contexto (`retriever`)  
2. Formatação do contexto (`format_docs`)  
3. Template do prompt  
4. Modelo de linguagem (`llm`)  
5. Parser de saída (`StrOutputParser`)


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## 7. Interagir com o Modelo

Agora podemos fazer perguntas sobre o documento PDF.  
Digite sua pergunta e o modelo responderá usando o contexto carregado.  
Para encerrar, basta digitar **`sair`**.


In [ ]:
print("Pronto para conversar com seu PDF. Digite 'sair' para encerrar.")
while True:
    question = input("Sua pergunta: ")
    if question.lower() == 'sair':
        break
    
    response = rag_chain.invoke(question)  # já é string
    display(Markdown(response))